Installing required huggingface libraries/modules

In [ ]:
pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=072c7feea1cc7bf5758ccdc714e0e76e5e0786262c82b0d741d8a5a6c13cc4b8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff85

In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00


Logging  to Hugging Face account so model can be shared and uploaded with community.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Mounting the Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


## TODO Recording:

- Create the Google Drive folder behind the scenes
- Open up Google Drive for the logged in user cloud.user@
- Show that you have a BBC folder under MyDrive
- Show the zip file already in that folder

In [ ]:
path_to_zip_file = "/content/drive/MyDrive/BBC/BBCNewsSummary.zip"

Extracting zip files into the drive

In [ ]:
import zipfile
with zipfile.ZipFile(path_to_zip_file, "r") as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/BBC")

## TODO Recording:

- Go to the BBC folder
- Show the subfolders
- Click through "News Articles" and show the categories
- Click through to a category and show the files


In [ ]:
import re

def extract(filepath):
    pattern = r"(\w+)/(\d+)\.txt$"

    category, file_id = re.search(pattern, str(filepath)).groups()

    with open(filepath, "r", encoding = "unicode_escape") as f:
        text = f.read()
        return category, file_id, text

Creating the dataframe of Articles and Summaries

In [ ]:
import pandas as pd
from pathlib import Path

dataset_path = Path("/content/drive/MyDrive/BBC/BBC News Summary/")

articles_data = list(map(extract, dataset_path.glob("News Articles/*/*.txt")))
summaries_data = list(map(extract, dataset_path.glob("Summaries/*/*.txt")))

articles_df = pd.DataFrame(articles_data, columns = ("Category", "ID", "Article"))
summaries_df = pd.DataFrame(summaries_data, columns = ("Category", "ID", "Summary"))

news_summary_df = articles_df.merge(summaries_df, how = "inner", on = ("Category", "ID"))

news_summary_df.head()

,Category,ID,Article,Summary
0,entertainment,289,Musicians to tackle US red tape\n\nMusicians' ...,Nigel McCune from the Musicians' Union said Br...
1,entertainment,262,"U2's desire to be number one\n\nU2, who have w...",But they still want more.They have to want to ...
2,entertainment,276,Rocker Doherty in on-stage fight\n\nRock singe...,"Babyshambles, which he formed after his acrimo..."
3,entertainment,060,Snicket tops US box office chart\n\nThe film a...,A Series of Unfortunate Events also stars Scot...
4,entertainment,074,Ocean's Twelve raids box office\n\nOcean's Twe...,"Ocean's Twelve, the crime caper sequel starrin..."


Take a look at an example of Article and its summary

In [ ]:
news_summary_df["Article"].loc[10]

"Actress Roberts takes spider role\n\nActress Julia Roberts will play the part of a spider in a new film version of children's classic Charlotte's Web.\n\nShe will voice Charlotte, who teams up with a girl to save their friend Wilbur the pig, in the story by EB White. The film - a mix of live action and animation - will be Roberts' first project since the birth of her twins, Hazel and Phinnaeus, two months ago. Oprah Winfrey will voice a goose, John Cleese will voice a sheep and Steve Buscemi a rat in the 2006 film.\n\nTen-year-old Dakota Fanning will play Fern, the girl at the centre of the story, in the film to be directed by 13 Going on 30 film-maker Gary Winick. Filming is due to begin in Melbourne, Australia, later this month. Charlotte's Web has sold 45 million copies since it was published in 1952. An animated version was made in 1973 but this will be the first live action film. The actor who will voice Wilbur the pig has yet to be revealed.\n"

In [ ]:
news_summary_df["Summary"].iloc[10]

"The film - a mix of live action and animation - will be Roberts' first project since the birth of her twins, Hazel and Phinnaeus, two months ago.An animated version was made in 1973 but this will be the first live action film.Actress Julia Roberts will play the part of a spider in a new film version of children's classic Charlotte's Web.Oprah Winfrey will voice a goose, John Cleese will voice a sheep and Steve Buscemi a rat in the 2006 film."

https://huggingface.co/docs/datasets/v1.2.0/loading_datasets.html#from-a-pandas-dataframe
Instantiating a Dataset object from pandas Dataframe.
Note that we are disabling default index,otherwise one addl column will be added as index

In [ ]:
from datasets import Dataset, DatasetDict

news_summary_ds = Dataset.from_pandas(news_summary_df, preserve_index = False)

news_summary_ds

Dataset({
    features: ['Category', 'ID', 'Article', 'Summary'],
    num_rows: 2225
})

Text cleaning function is defined

In [ ]:
def clean_txt(example):
    for txt in ["Article", "Summary"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("\\", " ")
       example[txt]  = example[txt].replace("/", " ")
       example[txt]  = example[txt].replace("\n"," ")
       example[txt]  = example[txt].replace("'s"," ")
       example[txt]  = example[txt].replace('"', ' ')
    return example

Then take a look at an example:

In [ ]:
cleaned_news_summary_ds = news_summary_ds.map(clean_txt)

cleaned_news_summary_ds

Map:   0%|          | 0/2225 [00:00<?, ? examples/s]

Dataset({
    features: ['Category', 'ID', 'Article', 'Summary'],
    num_rows: 2225
})

Observing one example of raw data

In [ ]:
news_summary_ds[0]

{'Category': 'entertainment',
 'ID': '289',
 'Article': 'Musicians to tackle US red tape\n\nMusicians\' groups are to tackle US visa regulations which are blamed for hindering British acts\' chances of succeeding across the Atlantic.\n\nA singer hoping to perform in the US can expect to pay $1,300 (Â£680) simply for obtaining a visa. Groups including the Musicians\' Union are calling for an end to the "raw deal" faced by British performers. US acts are not faced with comparable expense and bureaucracy when visiting the UK for promotional purposes.\n\nNigel McCune from the Musicians\' Union said British musicians are "disadvantaged" compared to their US counterparts. A sponsor has to make a petition on their behalf, which is a form amounting to nearly 30 pages, while musicians face tougher regulations than athletes and journalists. "If you make a mistake on your form, you risk a five-year ban and thus the ability to further your career," says Mr McCune.\n\n"The US is the world\'s bigges

Now observing cleaned up version of the same raw data

In [ ]:
cleaned_news_summary_ds[0]

{'Category': 'entertainment',
 'ID': '289',
 'Article': "musicians to tackle us red tape  musicians' groups are to tackle us visa regulations which are blamed for hindering british acts' chances of succeeding across the atlantic.  a singer hoping to perform in the us can expect to pay $1,300 (â£680) simply for obtaining a visa. groups including the musicians' union are calling for an end to the  raw deal  faced by british performers. us acts are not faced with comparable expense and bureaucracy when visiting the uk for promotional purposes.  nigel mccune from the musicians' union said british musicians are  disadvantaged  compared to their us counterparts. a sponsor has to make a petition on their behalf, which is a form amounting to nearly 30 pages, while musicians face tougher regulations than athletes and journalists.  if you make a mistake on your form, you risk a five-year ban and thus the ability to further your career,  says mr mccune.   the us is the world  biggest music market

## Preprocess

In [ ]:
from transformers import ForConditionalGeneration, PegasusTokenizer

Pegasus model tokenizer and model are instantiated

In [ ]:
PEGASUS_MODEL = "google/pegasus-cnn_dailymail"

tokenizer = PegasusTokenizer.from_pretrained(PEGASUS_MODEL)
model = PegasusForConditionalGeneration.from_pretrained(PEGASUS_MODEL)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pegasus tokenizer info can be seen

In [ ]:
tokenizer

PegasusTokenizer(name_or_path='google/pegasus-cnn_dailymail', vocab_size=96103, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>'

Pegasus Model can be observed

In [ ]:
model

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

Applying Model for summarization of a text

In [ ]:
EXAMPLE_TEXT_INDEX = 5

example_text = cleaned_news_summary_ds["Article"][EXAMPLE_TEXT_INDEX]

example_text

"'landmark movies' of 2004 hailed  us film professionals have declared fahrenheit 9 11 and the passion of the christ as two of the most significant cultural milestones of 2004.  the american film institute (afi) hailed mel gibson  biblical epic and michael moore  political documentary as inspiring national debate. it claimed both film-makers  tossed hollywood convention out the window . the institute also cited the death of actor marlon brando and the changing landscape of tv news in the us. in referring to marlon brando  death on 1 july at the age of 80, the 13-strong afi jury concluded  the art of screen acting has two chapters - 'before brando' and 'after brando'.  it credited the screen legend   raw hypnotic energy  and his ability to create characters like stanley kowalski and terry malloy  that will live forever in the annals of film history . the list also acknowledges key influences and trends in the world of film and broadcasting. among current trends, it highlighted the final

Summarisation on text is obtained. Setting truncation=True as some sentences are long enough to raise index error as maximum input sequence length of 1024 is getting exceeded

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model = PEGASUS_MODEL, truncation = True)
summary_txt = summarizer(example_text)

summary_txt

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'summary_text': 'the american film institute (afi) hailed mel gibson biblical epic and michael moore political documentary as inspiring national debate .<n>the institute also cited the death of actor marlon brando and the changing landscape of tv news in the us .<n> unsure of how the fcc will rule on an issue, the creative community has begun to self-censor their shows .'}]

Reference summary for that text is also obtained

In [ ]:
ref_txt = cleaned_news_summary_ds["Summary"][EXAMPLE_TEXT_INDEX]

ref_txt

"the institute also cited the death of actor marlon brando and the changing landscape of tv news in the us.the afi also questioned  the long-term viability of evening news broadcasts , in the light of 24-hour news channels and the internet.the list also acknowledges key influences and trends in the world of film and broadcasting.in referring to marlon brando  death on 1 july at the age of 80, the 13-strong afi jury concluded  the art of screen acting has two chapters - 'before brando' and 'after brando'.it its place, the afi fears, is a news landscape where  newscasters are more personalities than journalists  and balance and integrity are increasingly ignored.the american film institute (afi) hailed mel gibson  biblical epic and michael moore  political documentary as inspiring national debate."

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

Rouge score for that text summary is obtained

In [ ]:
summary_result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                               references = [ref_txt], use_stemmer = True)
summary_result

{'rouge1': 0.4615384615384615,
 'rouge2': 0.38341968911917096,
 'rougeL': 0.25641025641025644,
 'rougeLsum': 0.25641025641025644}

All article texts and summaries are defined in a list

In [ ]:
article_texts = cleaned_news_summary_ds["Article"]

article_summaries = cleaned_news_summary_ds["Summary"]

Generating summaries(zero shot) for first 50 rows only as it will takes a very long time in summarising whole set of examples

In [ ]:
from tqdm import tqdm

candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):
    candidate = summarizer(text)
    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [32:47<00:00, 39.34s/it]


Aggregated rouge scores are obtained

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.33804598715094203,
 'rouge2': 0.2355301381447623,
 'rougeL': 0.24334129098091573,
 'rougeLsum': 0.24498539503260527}

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

Minimum and maximum Rouge score indices are obtained to check best and worst summaries

In [ ]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(35, 22)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,Musicians' groups are to tackle us visa regula...,nigel mccune from the musicians' union said br...
1,"their new album, how to dismantle an atomic bo...",but they still want more.they have to want to ...
2,"babyshambles played for 5,000 fans at london b...","babyshambles, which he formed after his acrimo..."
3,"lemony snicket, starring jim carrey, took $30....",a series of unfortunate events also stars scot...
4,ocean twelve took $40.8m (£21m) in weekend tic...,"ocean twelve, the crime caper sequel starring..."


Taking a look at the highest and lowest rouge score case's predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(35, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(35, "Reference_summaries"))

Actual Summary
british actress antonia bernath makes her debut in kisna - the warrior poet .<n> marking a growing trend for non-indian stars to appear in bollywood films .<n> kisna has been shot simultaneously in english, with the english print shortened by an hour to accommodate the western audience .<n>the film even has a song composed in english by arrahman, the director of the popular west end musical, bombay dreams .
Reference Summary
british actress antonia bernath is making her debut in kisna - the warrior poet, marking a growing trend for non-indian stars to appear in bollywood films.directed by bollywood veteran subhash ghai, the film is set in the british raj and bernath plays an army general  daughter who falls in love with a rural boy, kisna. bollywood has truly gone global,  says taran adarsh, editor of indian film journal trade guide.and, many other bollywood stars have their eyes set on hollywood.actor and producer aamir khan  forthcoming venture the rising is set agains

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(22, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(22, "Reference_summaries"))

Actual Summary
mary poppins won two olivier awards for best choreography and best musical actress .<n>mel brooks' hit show triumphed in the battle for best new musical .<n>alan bennett the history boys was the big winner in the straight theatre categories, picking up three trophies .
Reference Summary
- best lighting design - his dark materials designed by paule constable at the olivier - best sound design - the woman in white designed by mick potter at the palace - best new opera - lady macbeth of mtsensk at the royal opera house - outstanding achievement in opera - thomas ades and the royal opera house for the world premiere of the tempest - best costume design - all  well that ends well designed by deirdre clancy at the gielgud - best revival - hamlet by william shakespeare at the old vic - best set design - his dark materials designed by giles cadle at the olivier - outstanding musical production - grand hotel at the donmar warehouse - best supporting role in a musical - conleth hi

Loading a  Barttokenizer to process `text` and `summary`:

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

Bart tokenizer and Bart large cnn model are instantiated

In [ ]:
BART_MODEL = "facebook/bart-large-cnn"

tokenizer = BartTokenizer.from_pretrained(BART_MODEL)
model = BartForConditionalGeneration.from_pretrained(BART_MODEL)

In [ ]:
tokenizer

BartTokenizer(name_or_path='facebook/bart-large-cnn', vocab_size=50265, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [ ]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

Summarisation on text is obtained. Setting truncation=True as some sentences are long enough to raise index error

In [ ]:
example_text

"'landmark movies' of 2004 hailed  us film professionals have declared fahrenheit 9 11 and the passion of the christ as two of the most significant cultural milestones of 2004.  the american film institute (afi) hailed mel gibson  biblical epic and michael moore  political documentary as inspiring national debate. it claimed both film-makers  tossed hollywood convention out the window . the institute also cited the death of actor marlon brando and the changing landscape of tv news in the us. in referring to marlon brando  death on 1 july at the age of 80, the 13-strong afi jury concluded  the art of screen acting has two chapters - 'before brando' and 'after brando'.  it credited the screen legend   raw hypnotic energy  and his ability to create characters like stanley kowalski and terry malloy  that will live forever in the annals of film history . the list also acknowledges key influences and trends in the world of film and broadcasting. among current trends, it highlighted the final

In [ ]:
summarizer = pipeline("summarization", model = BART_MODEL, truncation = True)

summary_txt = summarizer(example_text)

summary_txt

[{'summary_text': 'The american film institute (afi) hailed mel gibson  biblical epic and michael moore  political documentary as inspiring national debate. The institute also cited the death of actor marlon brando and the changing landscape of tv news in the us. The list also acknowledges key influences and trends in the world of film and broadcasting.'}]

In [ ]:
ref_txt

"the institute also cited the death of actor marlon brando and the changing landscape of tv news in the us.the afi also questioned  the long-term viability of evening news broadcasts , in the light of 24-hour news channels and the internet.the list also acknowledges key influences and trends in the world of film and broadcasting.in referring to marlon brando  death on 1 july at the age of 80, the 13-strong afi jury concluded  the art of screen acting has two chapters - 'before brando' and 'after brando'.it its place, the afi fears, is a news landscape where  newscasters are more personalities than journalists  and balance and integrity are increasingly ignored.the american film institute (afi) hailed mel gibson  biblical epic and michael moore  political documentary as inspiring national debate."

Rouge score for that text summary using Bart model is obtained

In [ ]:
summary_result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                               references = [ref_txt], use_stemmer = True)
summary_result

{'rouge1': 0.5744680851063829,
 'rouge2': 0.5591397849462365,
 'rougeL': 0.3723404255319149,
 'rougeLsum': 0.3723404255319149}

Generating summaries(zero shot) for first 50 rows only as it will takes a very long time in summarising whole set of examples

In [ ]:
candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(text)

    candidate_summaries.append(candidate[0]["summary_text"])

 32%|███▏      | 16/50 [05:08<10:02, 17.72s/it]

Aggregated rouge scores are obtained

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                           use_stemmer = True)
result_agg

Here case by case rouge scores are obtained.

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

Minimum and maximum Rouge score indices are obtained to check best and worst summaries

In [ ]:
result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

Taking a look at the highest and lowest rouge score case's predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(10, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(10, "Reference_summaries"))

"New York band scissor sisters have won a gig of the year award for their performance at this year's v festival. The award was voted for by listeners of virgin radio, which compiled a top 10 which was mostly dominated by newcomers on the music scene this year. The quirky disco-rock band beat the red hot chili peppers who came second for their hyde park performance in june."

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(46, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(46, "Reference_summaries"))

'new york band scissor sisters have won a gig of the year award for their performance at this year  v festival.the award was voted for by listeners of virgin radio, which compiled a top 10 which was mostly dominated by newcomers on the music scene this year.glasgow  franz ferdinand earned a place at number nine for their home-town performance in april.at number seven was oxfam  make fair trade gig at london  hammersmith apollo in october, which featured performances by rem, razorlight and coldplay  chris martin.'